In [7]:
import os
import numpy 
import librosa
import numpy as np
from keras.models import load_model
import python_speech_features as psf



#simple processing of the input signal
def getWavFeatures(wavPath):
    signal,rate = librosa.load(wavPath, sr=16000)
    if len(signal) >= 3 * 16000:
        signal = signal[0:int(3 * 16000)]
    else:
        signal = signal.tolist()
        for j in range(3 * 16000 - len(signal)):
            signal.append(0)
        signal = np.array(signal)
    feat = psf.logfbank(signal, samplerate=16000, nfilt=64)
    feat1 = psf.delta(feat, 1)
    feat2 = psf.delta(feat, 2)
    feat = feat.T[:, :, np.newaxis]
    feat1 = feat1.T[:, :, np.newaxis]
    feat2 = feat2.T[:, :, np.newaxis]
    fBank = np.concatenate((feat, feat1, feat2), axis=2)
    a = []
    a.append(fBank)
    fBank = np.array(a)
    # print(fBank.shape)
    return fBank

def predict(wav,h5):
    feat = getWavFeatures(wav)
    model = load_model(h5)
    pre = model.predict(feat)
    print(pre)
    
#predict("E:data_aishell/wav/test/S0764/BAC009S0764W0128.wav", "D:/桌面/gru1.h5")
#predict("E:data_aishell/wav/test/S0765/BAC009S0765W0128.wav", "D:/桌面/gru1.h5")
predict("E:data_aishell/wav/test/S0766/BAC009S0766W0130.wav", "D:/桌面/gru1.h5")
#predict("E:data_aishell/wav/test/S0767/BAC009S0767W0128.wav", "D:/桌面/gru1.h5")
#predict("E:data_aishell/wav/test/S0768/BAC009S0768W0128.wav", "D:/桌面/gru1.h5")
#predict("E:data_aishell/wav/test/S0769/BAC009S0769W0128.wav", "D:/桌面/gru1.h5")
#predict("E:data_aishell/wav/test/S0916/BAC009S0916W0128.wav", "D:/桌面/gru1.h5")

[[0.05655134 0.0556843  0.09655793 0.07553668 0.05672285 0.06919121
  0.0496043  0.04494899 0.04146263 0.04276752 0.03754447 0.03538237
  0.0495308  0.05015386 0.03949456 0.04081609 0.03487489 0.02607982
  0.0437035  0.05339182]]


In [ ]:
def getFeatures(filePath, batchSize=2, length=3, sampleRate=16000):
    numClasses = len(os.listdir(filePath))
    count = 0
    wavPath, wavLabel = getPathAndLabel(filePath)
    while True:
        cc = list(zip(wavPath, wavLabel))
        random.seed(10) #for reproductivity?
        #shuffle the shit
        random.shuffle(cc)
        wavPath[:], wavLabel[:] = zip(*cc)
        #print(wavLabel[:50])
        x = []
        y = []
        count = 0
        for index, wav in enumerate(wavPath):
            if count == batchSize:
                X = x
                Y = y
                # print(np.array(x).shape)
                X = np.array(X)  # (2, 64, 299, 3)
                Y = np.array(Y)
                Y = keras.utils.to_categorical(y, numClasses)
                # print()
                x = []
                y = []
                count = 0
                #print(np.argmax(Y))
                yield [X, Y]
                #returns feature matrix and labels

            else:
                signal, sampleRate = librosa.load(wav, sr=sampleRate)
                if len(signal) >= length * sampleRate:
                    signal = signal[0:int(3 * sampleRate)]
                # padding if less than 3 secs
                else:
                    signal = signal.tolist()
                    for j in range(length * sampleRate - len(signal)):
                        signal.append(0)
                    signal = np.array(signal)
                # print(len(signal))

                feat = psf.logfbank(signal, samplerate=16000, nfilt=64)
                feat1 = psf.delta(feat, 1)
                feat2 = psf.delta(feat, 2)
                feat = feat.T[:, :, np.newaxis]
                feat1 = feat1.T[:, :, np.newaxis]
                feat2 = feat2.T[:, :, np.newaxis]
                fBank = np.concatenate((feat, feat1, feat2), axis=2)
                x.append(fBank)
                y.append(wavLabel[index])
                count += 1
[X,Y] = getFeatures("E:/data_aishell/wav/test)